# **Global settings**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
from matplotlib.pyplot import rc_context
import scanpy as sc
import scrublet as scr
import scvelo as scv
import seaborn as sns
import scipy.io
import os
import dotplot
import dotplot.utils
import math
import gseapy as gp
from gseapy.plot import barplot, dotplot, gseaplot
from gseapy.scipalette import SciPalette
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap 

In [ ]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, figsize=(5, 5))

In [ ]:
plt.set_cmap('viridis')

<Figure size 500x500 with 0 Axes>

In [ ]:
scv.set_figure_params()

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc')

## **TF**

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.pyplot import rc_context
import scanpy as sc
import loompy as lp
import pyscenic
import glob
import os

### **loom文件**

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc/pySCENIC')

# file_in = '/disk213/xieqq/JINHUA138.sc/adata_CellLineage.h5ad'
file_in = '/disk213/xieqq/JINHUA138.sc/Epithelial_CellType.h5ad'
adata = sc.read_h5ad(file_in)
# adata = adata[adata.obs['CellType'].isin(['Enterocytes','Colonocytes'])]

# file_out = 'CellLineage_scenic.loom'
file_out = '/disk213/xieqq/JINHUA138.sc/pySCENIC_Epithelial/Epithelial_scenic.loom'
newdata = adata

# create basic row and column attributes for the loom file:
row_attrs = {"Gene": np.array(newdata.var_names)}
col_attrs = {"CellID": np.array(newdata.obs_names)}
lp.create(file_out, newdata.X.transpose(), row_attrs, col_attrs)

In [ ]:
## 细分
SEGMENT=adata.obs['SEGMENT'].cat.categories.tolist()
TIME=adata.obs['TIME'].cat.categories.tolist()
for segment in SEGMENT[:]:
    data = adata[adata.obs['SEGMENT'].isin([segment])]
    for time in TIME[:]:
        file_out = segment+'_'+time+'_scenic.loom'
        newdata = data[data.obs['TIME'].isin([time])]
        row_attrs = {"Gene": np.array(newdata.var_names)}
        col_attrs = {"CellID": np.array(newdata.obs_names)}
        lp.create(file_out, newdata.X.transpose(), row_attrs, col_attrs)

### **TF**

In [ ]:
# transcription factors list
# f_tfs = "/disk213/xieqq/sc/ipynb/pySCENIC/allTFs_Sus_scrofa.txt" # ss
# f_tfs = "/disk213/xieqq/sc/ipynb/pySCENIC/allTFs_hg38.txt" # human
# f_tfs = "/disk213/xieqq/sc/ipynb/pySCENIC/allTFs_mm.txt"   # mouse

In [ ]:
# source /disk211/public/anaconda3/bin/activate /disk211/public/anaconda3/envs/labBase 
cd /disk213/xieqq/JINHUA138.sc/pySCENIC
pyscenic grn --num_workers 5 --sparse --method grnboost2 --output CellLineage.grn.csv CellLineage_scenic.loom allTFs_hg38.txt
pyscenic ctx --num_workers 5 --output CellLineage.ctx.csv --expression_mtx_fname CellLineage_scenic.loom --mode "custom_multiprocessing" --annotations_fname motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl Epithelial.grn.csv hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
pyscenic aucell --num_workers 3 --output CellLineage_scenic_new.loom CellLineage_scenic.loom CellLineage.ctx.csv

In [ ]:
# source /disk211/public/anaconda3/bin/activate /disk211/public/anaconda3/envs/labBase 
cd /disk213/xieqq/JINHUA138.sc/pySCENIC_Epithelial
pyscenic grn --num_workers 5 --sparse --method grnboost2 --output Epithelial.grn.csv Epithelial_scenic.loom allTFs_hg38.txt
pyscenic ctx --num_workers 5 --output Epithelial.ctx.csv --expression_mtx_fname Epithelial_scenic.loom --mode "custom_multiprocessing" --annotations_fname motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl Epithelial.grn.csv hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
pyscenic aucell --num_workers 3 --output Epithelial_scenic_new.loom Epithelial_scenic.loom Epithelial.ctx.csv

In [ ]:
# source /disk211/public/anaconda3/bin/activate /disk211/public/anaconda3/envs/labBase 
arboreto_with_multiprocessing.py \
    Epithelial_scenic.loom \
    allTFs_hg38.txt \
    --method grnboost2 \
    --output Epithelial.grn.csv \
    --num_workers 10 \
    --seed 777